In [1]:
import numpy as np
import pandas as pd

In [2]:
bids = pd.read_csv('data/bids.csv')
trains = pd.read_csv('data/train.csv', usecols=['bidder_id', 'outcome'])
bids.head(2)

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url
0,0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,jewelry,phone0,9759243157894736,us,69.166.231.58,vasstdc27m7nks3
1,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,9759243157894736,in,50.201.125.84,jmqlhflrzwuay9c


In [3]:
bids.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7656334 entries, 0 to 7656333
Data columns (total 9 columns):
bid_id         7656334 non-null int64
bidder_id      7656334 non-null object
auction        7656334 non-null object
merchandise    7656334 non-null object
device         7656334 non-null object
time           7656334 non-null int64
country        7647475 non-null object
ip             7656334 non-null object
url            7656334 non-null object
dtypes: int64(2), object(7)
memory usage: 525.7+ MB


In [4]:
# 原始数据处理
bids['ip'] = bids['ip'].str.replace('.', '')
bids['country'] = bids['country'].fillna('unknown')

In [5]:
# 建立统计表
stat = pd.DataFrame(index=np.sort(bids['bidder_id'].unique()))

# bid_id

In [6]:
stat['bid_count'] = bids.groupby('bidder_id')['bid_id'].count()

id_auc = bids.groupby(['bidder_id', 'auction'])['bid_id'].count().groupby(level='bidder_id')
stat['bid_count_auc_mean'] = id_auc.mean()
stat['bid_count_auc_std'] = id_auc.std(ddof=0)
stat['bid_count_auc_max'] = id_auc.max()

id_ip = bids.groupby(['bidder_id', 'ip'])['bid_id'].count().groupby(level='bidder_id')
stat['bid_count_ip_mean'] = id_ip.mean()
stat['bid_count_ip_std'] = id_ip.std(ddof=0)
stat['bid_count_ip_max'] = id_ip.max()

id_url = bids.groupby(['bidder_id', 'url'])['bid_id'].count().groupby(level='bidder_id')
stat['bid_count_url_mean'] = id_url.mean()
stat['bid_count_url_std'] = id_url.std(ddof=0)
stat['bid_count_url_max'] = id_url.max()

In [7]:
# bot stat
bot_id = trains['bidder_id'][trains['outcome'] == 1].unique()
stat['bid_count'][stat.index.isin(bot_id)].sort_values().head(10)

bd0071b98d9479130e5c053a244fe6f1muj8h      1
91c749114e26abdb9a4536169f9b4580huern      1
74a35c4376559c911fdb5e9cfb78c5e4btqew      1
f35082c6d72f1f1be3dd23f949db1f577t6wd      1
7fab82fa5eaea6a44eb743bc4bf356b3tarle      1
efc61ad6db17265a4bac85f77b154ef414tpp      4
238808859d7752579a415e89a395500fh2o65     10
0f3ee77a46dd5a09ad20b7c3e54ec614sbpvb    137
40e00fd204d0ce97110032bfc47f423d0bgrd    148
d863897264a7e52d9c6a1be6dc453c6c9anz0    151
Name: bid_count, dtype: int64

In [8]:
# last 100 bid prop
auc_not_end = bids['auction'][bids['time'] == 9772885210526315].unique()
auc_sheet = bids[~bids['auction'].isin(auc_not_end)]
auc_stat = pd.DataFrame(index=auc_sheet['bidder_id'].sort_values().unique(), columns=bids['auction'].sort_values().unique(), data=np.nan)
auc_g = auc_sheet.groupby('auction')
for name, group in auc_g:
    if len(group) >= 100:
        last_100 = group.sort_values('time').iloc[-100:]['bidder_id'].value_counts()
        last_100p = last_100.div(last_100.sum())
        auc_stat.loc[last_100p.index, name] = last_100p

In [9]:
stat['bid_lst100_mean'] = np.nan
stat['bid_lst100_std'] =np.nan
stat['bid_lst100_max'] = np.nan
stat['bid_lst100_min'] = np.nan

stat['bid_lst100_mean'].loc[auc_stat.index] = auc_stat.mean(axis=1, skipna=True, numeric_only=True)
stat['bid_lst100_std'].loc[auc_stat.index] = auc_stat.std(axis=1, skipna=True, numeric_only=True)
stat['bid_lst100_max'].loc[auc_stat.index] = auc_stat.max(axis=1, skipna=True, numeric_only=True)
stat['bid_lst100_min'].loc[auc_stat.index] = auc_stat.min(axis=1, skipna=True, numeric_only=True)

C:\Users\carey\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [10]:
# first / last bid
auc_has_start = bids['auction'][bids['time'] == 9631916842105263].unique()
auc_not_end = bids['auction'][bids['time'] == 9772885210526315].unique()
auc_sheet_for_start = bids[~bids['auction'].isin(auc_has_start)]
auc_sheet_for_end = bids[~bids['auction'].isin(auc_not_end)]

stat['first_bid_count'] = np.nan
stat['last_bid_count'] = np.nan
stat['last_2nd_bid_count'] = np.nan

first_count = auc_sheet_for_start.sort_values('time').groupby('auction').first().groupby('bidder_id')['bid_id'].count()
last_count = auc_sheet_for_end.sort_values('time').groupby('auction').last().groupby('bidder_id')['bid_id'].count()
last_2nd = auc_sheet_for_end.sort_values('time').groupby('auction').nth(-2).groupby('bidder_id')['bid_id'].count()

stat['first_bid_count'].loc[first_count.index] = first_count
stat['last_bid_count'].loc[last_count.index] = last_count
stat['last_2nd_bid_count'].loc[last_2nd.index] = last_2nd

C:\Users\carey\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


# auction

In [12]:
def prop(series, need='arg'):
    v_count = series.value_counts()
    
    if need == 'arg':
        return v_count.index[0] # the one with largest value count
    
    v_count = v_count.values
    total = np.sum(v_count)
    p = v_count / total
    if need == 'max':
        return np.max(p)
    elif need == 'min':
        return np.min(p)
    elif need == 'mean':
        return np.mean(p)
    elif need == 'std':
        return np.std(p)
    
# order
def order_count(series):
    # caution: need sort time first
    ordered = series[series != series.shift()]
    return ordered.count()

In [13]:
id_g = bids.groupby('bidder_id')['auction']
stat['auc_nuni'] = id_g.nunique()
stat['auc_p_max'] = id_g.apply(prop, need='max')
stat['auc_p_min'] = id_g.apply(prop, need='min')
stat['auc_p_std'] = id_g.apply(prop, need='std')

id_ip = bids.groupby(['bidder_id', 'ip'])['auction'].nunique().groupby(level='bidder_id')
stat['auc_ip_nuni_mean'] = id_ip.mean()
stat['auc_ip_nuni_std'] = id_ip.std(ddof=0)
stat['auc_ip_nuni_max'] = id_ip.max()

id_ip = bids.groupby(['bidder_id', 'ip'])['auction'].apply(prop, need='max').groupby(level='bidder_id')
stat['auc_ip_count_max_max'] = id_ip.max()
stat['auc_ip_count_max_mean'] = id_ip.mean()
stat['auc_ip_count_max_std'] = id_ip.std(ddof=0)

id_ip = bids.groupby(['bidder_id', 'ip'])['auction'].apply(prop, need='mean').groupby(level='bidder_id')
stat['auc_ip_count_mean_mean'] = id_ip.mean()
stat['auc_ip_count_mean_std'] = id_ip.std(ddof=0)

id_ip = bids.groupby(['bidder_id', 'ip'])['auction'].apply(prop, need='std').groupby(level='bidder_id')
stat['auc_ip_count_std_mean'] = id_ip.mean()

id_url = bids.groupby(['bidder_id', 'url'])['auction'].nunique().groupby(level='bidder_id')
stat['auc_url_nuni_mean'] = id_auc.mean()
stat['auc_url_nuni_std'] = id_auc.std(ddof=0)
stat['auc_url_nuni_max'] = id_auc.max()

id_url = bids.groupby(['bidder_id', 'url'])['auction'].apply(prop, need='max').groupby(level='bidder_id')
stat['auc_url_count_max_mean'] = id_auc.mean()
stat['auc_url_count_max_std'] = id_auc.std(ddof=0)
stat['auc_url_count_max_max'] = id_auc.max()

id_url = bids.groupby(['bidder_id', 'url'])['auction'].apply(prop, need='mean').groupby(level='bidder_id')
stat['auc_url_count_mean_mean'] = id_auc.mean()
stat['auc_url_count_mean_std'] = id_auc.std(ddof=0)

id_url = bids.groupby(['bidder_id', 'url'])['auction'].apply(prop, need='std').groupby(level='bidder_id')
stat['auc_url_count_std_mean'] = id_auc.mean()

print('finish')

finish


In [14]:
# auc order
stat['auc_order'] = bids.sort_values('time').groupby('bidder_id')['auction'].apply(order_count)

# merchandise

In [16]:
id_g = bids.groupby('bidder_id')['merchandise']
stat['merch_nuni'] = id_g.nunique()
stat['merch_arg'] = id_g.apply(prop)
stat['merch_p_max'] = id_g.apply(prop, need='max')
stat['merch_p_min'] = id_g.apply(prop, need='min')
stat['merch_p_mean'] = id_g.apply(prop, need='mean')
stat['merch_p_std'] = id_g.apply(prop, need='std')

# device

In [17]:
id_g = bids.groupby('bidder_id')['device']
stat['dev_nuni'] = id_g.nunique()
stat['dev_p_max'] = id_g.apply(prop, need='max')
stat['dev_p_min'] = id_g.apply(prop, need='min')
stat['dev_p_mean'] = id_g.apply(prop, need='mean')
stat['dev_p_std'] = id_g.apply(prop, need='std')

# time

In [18]:
id_g = bids.groupby('bidder_id')['time']
stat['t_max'] = id_g.max()
stat['t_min'] = id_g.min()
stat['t_range'] = id_g.apply(lambda x: x.max() - x.min())

In [19]:
# time diff
def dif(series, need='mean'):
    if len(series) <= 1:
        return 0
    diff = series.sort_values().diff().dropna()
    if need == 'mean':
        return diff.mean()
    if need == 'std':
        return diff.std(ddof=0)
    if need == 'max':
        return diff.max()
    if need == 'min':
        return diff.min()
    if need == 'q25':
        return diff.quantile(q=0.25)
    if need == 'q50':
        return diff.quantile(q=0.5)
    if need == 'q75':
        return diff.quantile(q=0.75)

In [20]:
## per id
id_g = bids.groupby('bidder_id')['time']
stat['t_dif_mean'] = id_g.apply(dif)
stat['t_dif_std'] = id_g.apply(dif, need='std')
stat['t_dif_max'] = id_g.apply(dif, need='max')
stat['t_dif_min'] = id_g.apply(dif, need='min')
stat['t_dif_q25'] = id_g.apply(dif, need='q25')
stat['t_dif_q50'] = id_g.apply(dif, need='q50')
stat['t_dif_q75'] = id_g.apply(dif, need='q75')

In [21]:
## per id per auc
id_auc = bids.groupby(['bidder_id', 'auction'])['time'].apply(lambda x: x.max() - x.min()).groupby(level='bidder_id')
stat['t_range_auc_mean'] = id_auc.mean()
stat['t_range_auc_std'] = id_auc.std(ddof=0)

id_auc = bids.groupby(['bidder_id', 'auction'])['time'].apply(dif, need='min').groupby(level='bidder_id')
stat['t_dif_auc_min_min'] = id_auc.min()
stat['t_dif_auc_min_mean'] = id_auc.mean()
stat['t_dif_auc_min_std'] = id_auc.std(ddof=0)


id_auc = bids.groupby(['bidder_id', 'auction'])['time'].apply(dif, need='max').groupby(level='bidder_id')
stat['t_dif_auc_max_max'] = id_auc.max()
stat['t_dif_auc_max_mean'] = id_auc.mean()
stat['t_dif_auc_max_std'] = id_auc.std(ddof=0)

id_auc = bids.groupby(['bidder_id', 'auction'])['time'].apply(dif, need='mean').groupby(level='bidder_id')
stat['t_dif_auc_mean_mean'] = id_auc.mean()
stat['t_dif_auc_mean_std'] = id_auc.std(ddof=0)

id_auc = bids.groupby(['bidder_id', 'auction'])['time'].apply(dif, need='std').groupby(level='bidder_id')
stat['t_dif_auc_std_mean'] = id_auc.mean()

print('finish')

finish


In [22]:
# t resp
bids['t_resp'] = np.nan
auc_groups = bids.groupby('auction')
for name, group in auc_groups:
    group = group.sort_values('time')[['bidder_id', 'time']]
    group['t_dif'] = group['time'].diff()
    t_resp = group['t_dif'][group['bidder_id'] != group['bidder_id'].shift()].dropna()
    bids['t_resp'].loc[t_resp.index] = t_resp
    
print('finish')

C:\Users\carey\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


finish


In [23]:
id_g = bids.groupby('bidder_id')['t_resp']

stat['t_resp_min'] = id_g.min()
stat['t_resp_max'] = id_g.max()
stat['t_resp_mean'] = id_g.apply(lambda x: x.dropna().mean())
stat['t_resp_std'] = id_g.apply(lambda x: x.dropna().std(ddof=0))
stat['t_resp_median'] = id_g.apply(lambda x: x.dropna().median())

In [24]:
id_auc = bids.groupby(['bidder_id', 'auction'])['t_resp'].apply(lambda x: x.dropna().mean()).groupby(level='bidder_id')
stat['t_resp_auc_mean_mean'] = id_auc.apply(lambda x: x.dropna().mean())
stat['t_resp_auc_mean_std'] = id_auc.apply(lambda x: x.dropna().std(ddof=0))

id_auc = bids.groupby(['bidder_id', 'auction'])['t_resp'].apply(lambda x: x.dropna().median()).groupby(level='bidder_id')
stat['t_resp_auc_median_mean'] = id_auc.apply(lambda x: x.dropna().mean())
stat['t_resp_auc_median_std'] = id_auc.apply(lambda x: x.dropna().std(ddof=0))

id_auc = bids.groupby(['bidder_id', 'auction'])['t_resp'].apply(lambda x: x.dropna().std()).groupby(level='bidder_id')
stat['t_resp_auc_std_mean'] = id_auc.apply(lambda x: x.dropna().mean())

In [25]:
# t dup
id_groups = bids.groupby('bidder_id')
stat['t_dup_count'] = id_groups['time'].apply(lambda x: x.duplicated(keep=False).count())

In [26]:
# t slices
ts_1 = np.linspace(9631916842105263, 9645558894736842, 85)
ts_2 = np.linspace(9695580000000000, 9709222052631578, 85)
ts_3 = np.linspace(9759243157894736, 9772885210526315, 85)
ts = np.append(ts_1, [ts_2, ts_3])

stat_ts = pd.DataFrame(index=bids['bidder_id'].unique())

for i in range(len(ts) - 1):
    ts_sheet = bids[bids['time'].between(ts[i], ts[i+1])]
    ts_bid = ts_sheet.groupby('bidder_id')['bid_id'].count()
    stat_ts['ts_' + str(i)] = np.nan
    stat_ts['ts_' + str(i)].loc[ts_bid.index] = ts_bid

print('finish')

finish


In [27]:
stat['tl_mean'] = stat_ts.mean(axis=1, skipna=True, numeric_only=True)
stat['tl_std'] = stat_ts.std(axis=1, ddof=0, skipna=True, numeric_only=True)
stat['tl_max'] = stat_ts.max(axis=1, skipna=True, numeric_only=True)
stat['tl_min'] = stat_ts.min(axis=1, skipna=True, numeric_only=True)
stat['tl_q25'] = stat_ts.quantile(q=0.25, axis=1, numeric_only=True)
stat['tl_q50'] = stat_ts.quantile(q=0.5, axis=1, numeric_only=True)
stat['tl_q75'] = stat_ts.quantile(q=0.75, axis=1, numeric_only=True)

# country

In [28]:
id_g = bids.groupby('bidder_id')['country']
stat['cty_nuni'] = id_g.nunique()
stat['cty_arg'] = id_g.apply(prop)
stat['cty_p_max'] = id_g.apply(prop, need='max')
stat['cty_p_min'] = id_g.apply(prop, need='min')
stat['cty_p_mean'] = id_g.apply(prop, need='mean')
stat['cty_p_std'] = id_g.apply(prop, need='std')
stat['cty_order'] = bids.sort_values('time').groupby('bidder_id')['country'].apply(order_count)

id_auc = bids.groupby(['bidder_id', 'auction'])['country'].apply(prop, need='max').groupby(level='bidder_id')
stat['cty_auc_p_max_max'] = id_auc.max()
stat['cty_auc_p_max_mean'] = id_auc.mean()
stat['cty_auc_p_max_std'] = id_auc.std(ddof=0)

id_auc = bids.groupby(['bidder_id', 'auction'])['country'].apply(prop, need='min').groupby(level='bidder_id')
stat['cty_auc_p_min_min'] = id_auc.min()
stat['cty_auc_p_min_mean'] = id_auc.mean()
stat['cty_auc_p_min_std'] = id_auc.std(ddof=0)

id_auc = bids.groupby(['bidder_id', 'auction'])['country'].apply(prop, need='mean').groupby(level='bidder_id')
stat['cty_auc_p_mean_mean'] = id_auc.mean()
stat['cty_auc_p_mean_std'] = id_auc.std(ddof=0)

id_auc = bids.groupby(['bidder_id', 'auction'])['country'].apply(prop, need='std').groupby(level='bidder_id')
stat['cty_auc_p_std_mean'] = id_auc.mean()

print('finish')

finish


# ip

In [29]:
id_g = bids.groupby('bidder_id')['ip']
stat['ip_nuni'] = id_g.nunique()
stat['ip_p_max'] = id_g.apply(prop, need='max')
stat['ip_p_min'] = id_g.apply(prop, need='min')
stat['ip_p_mean'] = id_g.apply(prop, need='mean')
stat['ip_p_std'] = id_g.apply(prop, need='std')

id_auc = bids.groupby(['bidder_id', 'auction'])['ip'].apply(prop, need='max').groupby(level='bidder_id')
stat['ip_auc_p_max_max'] = id_auc.max()
stat['ip_auc_p_max_mean'] = id_auc.mean()
stat['ip_auc_p_max_std'] = id_auc.std(ddof=0)

id_auc = bids.groupby(['bidder_id', 'auction'])['ip'].apply(prop, need='min').groupby(level='bidder_id')
stat['ip_auc_p_min_min'] = id_auc.min()
stat['ip_auc_p_min_mean'] = id_auc.mean()
stat['ip_auc_p_min_std'] = id_auc.std(ddof=0)

id_auc = bids.groupby(['bidder_id', 'auction'])['ip'].apply(prop, need='mean').groupby(level='bidder_id')
stat['ip_auc_p_mean_mean'] = id_auc.mean()
stat['ip_auc_p_mean_std'] = id_auc.std(ddof=0)

id_auc = bids.groupby(['bidder_id', 'auction'])['ip'].apply(prop, need='std').groupby(level='bidder_id')
stat['ip_auc_p_std_mean'] = id_auc.mean()

print('finish')

finish


In [30]:
# ip order
id_g = bids.sort_values('time').groupby('bidder_id')['ip']
stat['ip_order'] = id_g.apply(order_count)

# url

In [31]:
id_g = bids.groupby('bidder_id')['url']
stat['url_nuni'] = id_g.nunique()
stat['url_p_max'] = id_g.apply(prop, need='max')
stat['url_p_min'] = id_g.apply(prop, need='min')
stat['url_p_mean'] = id_g.apply(prop, need='mean')
stat['url_p_std'] = id_g.apply(prop, need='std')

id_auc = bids.groupby(['bidder_id', 'auction'])['url'].apply(prop, need='max').groupby(level='bidder_id')
stat['url_auc_p_max_max'] = id_auc.max()
stat['url_auc_p_max_mean'] = id_auc.mean()
stat['url_auc_p_max_std'] = id_auc.std(ddof=0)

id_auc = bids.groupby(['bidder_id', 'auction'])['url'].apply(prop, need='min').groupby(level='bidder_id')
stat['url_auc_p_min_min'] = id_auc.min()
stat['url_auc_p_min_mean'] = id_auc.mean()
stat['url_auc_p_min_std'] = id_auc.std(ddof=0)

id_auc = bids.groupby(['bidder_id', 'auction'])['url'].apply(prop, need='mean').groupby(level='bidder_id')
stat['url_auc_p_mean_mean'] = id_auc.mean()
stat['url_auc_p_mean_std'] = id_auc.std(ddof=0)

id_auc = bids.groupby(['bidder_id', 'auction'])['url'].apply(prop, need='std').groupby(level='bidder_id')
stat['url_auc_p_std_mean'] = id_auc.mean()

print('finish')

finish


In [32]:
# url order
id_g = bids.sort_values('time').groupby('bidder_id')['url']
stat['url_order'] = id_g.apply(order_count)

In [34]:
filename = 'stat_0602'

stat.to_csv('stat/' + filename + '.csv', index_label='bidder_id')